In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import json

# Data Wrangling
import h5py
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import tables
import flammkuchen as fl

# Computation
from scipy.interpolate import interp1d
from datetime import datetime
import math
import itertools
from scipy.signal.signaltools import correlate
import pickle
from tqdm import tqdm


### Import custom functions

In [22]:
# Megabouts:
from megabouts.pipeline.cfg import ConfigTrajPreprocess,ConfigTailPreprocess,ConfigTailSegmentation,ConfigClassification
#Megabouts pipeline 
from megabouts.pipeline.full_tracking import PipelineFullTracking,PipelineFullTracking_Result


    TODO: check for usages of functions and clean -> maybe add more functions here

In [21]:

# my own utils
from utils_motato import compute_angle_between_vect_tail,compute_angle_between_vect,exptrapolate_segments
from utils_motato import tail_angles, eye_preprocess,fin_preprocess
from utils_motato import reduce_to_pi, resample_behavior, match_times, reconstruct
from utils_motato import cluster_colors, clust_colors

In [5]:
from feature_extractuion_helpers import nanzscore
from preprocess_helpers import reduce_to_pi
from preprocess_helpers import rolling_window
from preprocess_helpers import mid_head, midpoint
from preprocess_helpers import compute_tailsum, clean_tail, tail_angles
from preprocess_helpers import fin_preprocess

## Set up paths

In [6]:
# master_path = Path(r"\\portulab.synology.me\data\Kata\230307_visstim_2D")
#master_path = Path(r"\\portulab.synology.me\data\Kata\231106_drunk_fish\Control")
#master_path = Path(r"\\portulab.synology.me\data\Kata\231106_drunk_fish\Drunk\test")
# master_path = Path(r"Z:\Kata\230307_visstim_2D")
master_path = Path(r"\\portulab.synology.me\data\Kata\221209_DLC_set3")
fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths

[]

In [7]:
# out_path = Path(r"\\portulab.synology.me\data\Kata\21042024_data\230307_visstim_2D_")
# out_path = Path(r"\\portulab.synology.me\data\Kata\21042024_data\231106_drunk_fish_control")
# out_path = Path(r"\\portulab.synology.me\data\Kata\21042024_data\231106_drunk_fish_")
out_path = Path(r"\\portulab.synology.me\data\Kata\21042024_data\221209_DLC_set3_")
out_path

WindowsPath('//portulab.synology.me/data/Kata/21042024_data/221209_DLC_set3_')

In [8]:
print ("{} videos found".format(len(fish_paths)))

0 videos found


In [9]:
# set up params that dont change 
fps_new = 200
fps = 200


### Setup pipeline once

In [14]:

cfg_tail_preprocess = ConfigTailPreprocess(fps=fps_new,num_pcs=4,limit_na_ms=100,
                                           baseline_method='slow',
                                           baseline_params={'fps':700})

cfg_traj_preprocess = ConfigTrajPreprocess(fps=fps_new,freq_cutoff_min=10,beta=4)


cfg_segment = ConfigTailSegmentation(fps=fps_new,
                                     tail_speed_filter_ms=100,tail_speed_boxcar_filter_ms=14,tail_speed_thresh_std=2.1,
                                     min_bout_duration_ms=85,bout_duration_ms=200)

cfg_classify = ConfigClassification(fps=fps_new,
                                      margin_before_peak_ms=40,
                                      bout_duration_ms=200,
                                      augment_min_delay_ms=-20,
                                      augment_max_delay_ms=20,
                                      augment_step_delay_ms=4,
                                      feature_weight= np.array([1.6]*7+[0,0,1]),
                                      N_kNN=10
                                      )

pipeline = PipelineFullTracking(cfg_tail_preprocess,
                                cfg_traj_preprocess,
                                cfg_segment,
                                cfg_classify,
                                load_training=True)


(46710, 10, 49)
TimeStep:40,OriginalPeakLoc:9,TargetPeak:8
augmentation_delays:[-4 -3 -2 -1  0  1  2  3]
IdSt:5
IdSt:4
IdSt:3
IdSt:2
IdSt:1
IdSt:0
IdSt:-1
IdSt:-2


# Run single fish megabouts and pool with other data

    TODO: take out thresh
    TODO: put actual templates matched in and not general library

In [11]:
N_seg = 10

fish =0
fish_path = fish_paths[fish]
fish_id = os.path.basename(fish_path)
fish_path
print ('working on :', fish_path, fish_id)

IndexError: list index out of range

In [12]:
filename = list(fish_path.glob('*316000.h5*'))[0]
df= pd.read_hdf(filename,  header=[1, 2], index_col=0)
df = df['DLC_resnet50_dlc_2Dec12shuffle1_316000']
print(f'{df.shape[0]/(fps*60)} minutes at {fps} fps')

#Extract angles
body_x = df.body.values[:, 0].astype('float')
body_y = df.body.values[:, 1].astype('float')
tail_x_col = [f'tail_{i}' for i in range(5)]
tail_y_col = [f'tail_{i}' for i in range(5)]
tail_x = np.array([df[x].iloc[:, 0].values.astype('float') for x in tail_x_col])
tail_y = np.array([df[x].iloc[:, 1].values.astype('float') for x in tail_y_col])

new_tail_x, new_tail_y = clean_tail(tail_x, tail_y, rolling=False,  wnd=5, thresh=100)
tail_x_10, tail_y_10 = exptrapolate_segments(new_tail_x, new_tail_y, N_seg)

tail_angle, body_angle = tail_angles(tail_x_10, tail_y_10, body_x, body_y, N_seg)
tailsum = compute_tailsum(tail_angle)



2.28525 minutes at 200 fps


NameError: name 'compute_angle_between_vect_tail' is not defined

In [ ]:
# calculate fins
mid_headx, mid_heady = mid_head(df)
left_fin_vect, right_fin_vect, left_fin_angle, right_fin_angle = fin_preprocess(df, body_angle, mid_headx, mid_heady, tail_x_10, tail_y_10)


In [18]:
# Load eye stuff
eye_angles = fl.load(fish_path/'eye_angles.h5')['eye_angles'] #for the hdf5 way of saving dict needs ['eye_angles']
vergence = fl.load(fish_path/'eye_rot.h5')['eye_rot']
rotation_eye = fl.load(fish_path/'eye_verg.h5')['eye_verg']
eye_coords = fl.load(fish_path/'eye_coords.h5')['eye_coords']

left_eye_angle = np.rad2deg(eye_angles[:,0])
right_eye_angle = np.rad2deg(eye_angles[:,1])
rotation_eye = np.rad2deg(rotation_eye)

In [19]:
#Run pipeline object
x,y = np.zeros_like(body_angle),np.zeros_like(body_angle)
res = pipeline.run(tail_angle,x,y,body_angle)

Preprocessing Tail and Trajectory
Segmentation
Classification


In [34]:
#  get only real templates
id_nearest_template = res.classification.id_nearest_template_aligned
tail_templates = pipeline.knn_training_dataset_augmented.tail
tails_nearest = tail_templates[id_nearest_template]

In [39]:

#save results per fish
res_dict = {'fish_id': fish_id,
        'body_angle': np.asarray(body_angle),
        'tail_angle': np.asarray(tail_angle),
        'eye_angles': np.asarray([left_eye_angle, right_eye_angle]), 
        'vergence': np.asarray(vergence),
        'rotation': np.asarray(rotation_eye), 
        'fin_angles': np.asarray([left_fin_angle, right_fin_angle]),
        'eye_coords': np.asarray(eye_coords),

        'templates_labels': pipeline.knn_training_dataset_augmented.labels, 
        'tail_and_traj_array' : res.tail_and_traj_array,
        'bout_category' : res.classification.bout_category,
        'bout_category_ts': res.bout_category_ts,
        'proba' : res.classification.proba,
        'outlier' : res.classification.outlier_score,
        'tail_speed' : res.smooth_tail_speed,
        'onset_shift': res.classification.onset_shift,
        'segments_on': np.asarray(res.segments.onset),
        'segments_off': np.asarray(res.segments.offset),
        'id_nearest_template' : res.classification.id_nearest_template_aligned,
        'tails_nearest' : tail_templates[id_nearest_template],
        'clean_data_tail': res.tracking_data_clean.tail_angle, 
       }


hf = h5py.File(out_path /'{}_megabouts_res.h5'.format(fish_id), 'w')
dict_group = hf.create_group('data')
for k, v in res_dict.items():
    dict_group[k] = v
hf.close()

In [ ]:
# #  Save tail template array once

# tail_templates = pipeline.knn_training_dataset_augmented.tail

# hf = h5py.File(out_path_path_path /'tail_templates.h5', 'w')
# dict_group = hf.create_group('templates')
# for k, v in res_dict.items():
#     dict_group[k] = v
# hf.close()

In [17]:
out_path =  Path(r"\\portulab.synology.me\data\Kata\Processed_Data\12052024_visstim_round_")

In [18]:
master_path = Path(r"\\portulab.synology.me\data\Kata\Data\12052024_visstim_round")
fish_paths = list(master_path.glob('*f[0-9]*'))
fish_paths

[WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f0'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f1'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f10'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f11'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f2'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f3'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f4'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f5'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f6'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f7'),
 WindowsPath('//portulab.synology.me/data/Kata/Data/12052024_visstim_round/240512_f8'),
 WindowsPath('//portulab.synol

# Run Pipeline:

In [ ]:
N_seg = 10


for ind, fish_path in enumerate(tqdm(fish_paths)):
    fish_id = os.path.basename(fish_path)
    print ('working on :', fish_path, fish_id)

    filename = list(fish_path.glob('*316000.h5*'))[0]
    df= pd.read_hdf(filename,  header=[1, 2], index_col=0)
    df = df['DLC_resnet50_dlc_2Dec12shuffle1_316000']
    print(f'{df.shape[0]/(fps*60)} minutes at {fps} fps')
    
    #Extract angles
    body_x = df.body.values[:, 0].astype('float')
    body_y = df.body.values[:, 1].astype('float')
    tail_x_col = [f'tail_{i}' for i in range(5)]
    tail_y_col = [f'tail_{i}' for i in range(5)]
    tail_x = np.array([df[x].iloc[:, 0].values.astype('float') for x in tail_x_col])
    tail_y = np.array([df[x].iloc[:, 1].values.astype('float') for x in tail_y_col])
    
    new_tail_x, new_tail_y = clean_tail(tail_x, tail_y, rolling=False,  wnd=5, thresh=100)
    tail_x_10, tail_y_10 = exptrapolate_segments(new_tail_x, new_tail_y, N_seg)
    
    tail_angle, body_angle = tail_angles(tail_x_10, tail_y_10, body_x, body_y, N_seg)
    tailsum = compute_tailsum(tail_angle)

    # calculate fins
    mid_headx, mid_heady = mid_head(df)
    left_fin_vect, right_fin_vect, left_fin_angle, right_fin_angle = fin_preprocess(df, body_angle, mid_headx, mid_heady, tail_x_10, tail_y_10)

    # Load eye stuff
    eye_angles = fl.load(fish_path/'eye_angles.h5')['eye_angles'] #for the hdf5 way of saving dict needs ['eye_angles']
    vergence = fl.load(fish_path/'eye_rot.h5')['eye_rot']
    rotation_eye = fl.load(fish_path/'eye_verg.h5')['eye_verg']
    eye_coords = fl.load(fish_path/'eye_coords.h5')['eye_coords']
    
    left_eye_angle = np.rad2deg(eye_angles[:,0])
    right_eye_angle = np.rad2deg(eye_angles[:,1])
    rotation_eye = np.rad2deg(rotation_eye)

    # Run pipeline object
    x,y = np.zeros_like(body_angle),np.zeros_like(body_angle)
    res = pipeline.run(tail_angle,x,y,body_angle)

    #  get only real templates
    id_nearest_template = res.classification.id_nearest_template_aligned
    tail_templates = pipeline.knn_training_dataset_augmented.tail
    tails_nearest = tail_templates[id_nearest_template]
    
    #save results per fish
    res_dict = {'fish_id': fish_id,
            'body_angle': np.asarray(body_angle),
            'tail_angle': np.asarray(tail_angle),
            'eye_angles': np.asarray([left_eye_angle, right_eye_angle]), 
            'vergence': np.asarray(vergence),
            'rotation': np.asarray(rotation_eye), 
            'fin_angles': np.asarray([left_fin_angle, right_fin_angle]),
            'eye_coords': np.asarray(eye_coords),
    
            'templates_labels': pipeline.knn_training_dataset_augmented.labels, 
            'tail_and_traj_array' : res.tail_and_traj_array,
            'bout_category' : res.classification.bout_category,
            'bout_category_ts': res.bout_category_ts,
            'proba' : res.classification.proba,
            'outlier' : res.classification.outlier_score,
            'tail_speed' : res.smooth_tail_speed,
            'onset_shift': res.classification.onset_shift,
            'segments_on': np.asarray(res.segments.onset),
            'segments_off': np.asarray(res.segments.offset),
            'id_nearest_template' : res.classification.id_nearest_template_aligned,
            'tails_nearest' : np.asarray(tails_nearest),
            'clean_data_tail': res.tracking_data_clean.tail_angle, 
           }
    
    
    hf = h5py.File(out_path /'{}_megabouts_res.h5'.format(fish_id), 'w')
    dict_group = hf.create_group('data')
    for k, v in res_dict.items():
        dict_group[k] = v
    hf.close()



  0%|                                                                                           | 0/12 [00:00<?, ?it/s]

working on : \\portulab.synology.me\data\Kata\Data\12052024_visstim_round\240512_f0 240512_f0
7.802333333333333 minutes at 200 fps
corr fins
Preprocessing Tail and Trajectory
Segmentation
Classification
